# This is the new Extractor Code

In [3]:
import requests
import json

url = 'xxx'
user = 'xxx'
pwd = 'xxx'

response = requests.get(url, auth=(user, pwd))

data = response.json()

tickets = data['tickets']


def extract_one(blob, *keys, transformer=None):
    """
    This function will extract the value at the nested "path" supplied
    by the *keys arguments.

    if transformer is a function, than this function will be run on the
    item located at the specified path.
    """
    # Make sure the user passes at least one key to extract
    if len(keys) == 0:
        raise ValueError('You must supply at least one key')

    # If only one key passed, then just grab the entry at that key
    if len(keys) == 1:
        # BUT... if a tranformer is supplied, run the transformer on the value at that key
        if transformer:
            return transformer(blob.get(keys[0]))
        else:
            return blob.get(keys[0])
    # If there is more than one key, then you know you are not yet at the address you need
    # Descend the path tree to the next key in the "address list"
    else:
        sub_blob = blob[keys[0]]

        # Recursively call this same function with the remaining addresses on the chain
        return extract_one(sub_blob, *keys[1:], transformer=transformer)


def get_total_seconds(iterable):
    """
    This is a special transformer function that we will use to extract the total seconds
    from the blob.  You can pass transformer functions to do special logic that is
    required other than simply pulling out nested keys.
    """
    custom_field_id = 360016643492

    if not isinstance(iterable, list):
        return iterable

    # I know this input will be a list, so just iterate over it
    for item in iterable:
        # If I've found the item id I care about, return it
        if item['id'] == custom_field_id:
            # Grab the value and transform it to an integer if it's a string
            out = item['value']
            if isinstance(out, str):
                out = int(out)
            # return the value
            return out

    # The only way you make it here is if this record didn't find the custom field you want
    raise ValueError(f"Uh oh! I couldn't find the custom field id {custom_field_id}")


def extract_from_ticket(ticket):
    return {
        'assignee_id': extract_one(ticket, 'assignee_id'),
        'url': extract_one(ticket, 'url'),
        'id': extract_one(ticket, 'id'),
        'status': extract_one(ticket, 'status'),
        'created_at': extract_one(ticket, 'created_at'),
        'total_seconds': extract_one(ticket, 'custom_fields', transformer=get_total_seconds),
        'satisfaction_rating': extract_one(ticket, 'satisfaction_rating', 'score')
    }

from pprint import pprint
records = [extract_from_ticket(t) for t in tickets]
pprint(records)


[{'assignee_id': 380454819451,
  'created_at': '2021-05-18T12:54:23Z',
  'id': 50822,
  'satisfaction_rating': 'offered',
  'status': 'closed',
  'total_seconds': 1007,
  'url': 'https://ambition.zendesk.com/api/v2/tickets/50822.json'},
 {'assignee_id': 380454819451,
  'created_at': '2021-05-17T07:29:13Z',
  'id': 50815,
  'satisfaction_rating': 'offered',
  'status': 'closed',
  'total_seconds': 2207,
  'url': 'https://ambition.zendesk.com/api/v2/tickets/50815.json'},
 {'assignee_id': 389367924132,
  'created_at': '2021-05-10T07:58:25Z',
  'id': 50755,
  'satisfaction_rating': 'offered',
  'status': 'closed',
  'total_seconds': 993,
  'url': 'https://ambition.zendesk.com/api/v2/tickets/50755.json'},
 {'assignee_id': 380454819451,
  'created_at': '2021-05-07T08:14:14Z',
  'id': 50740,
  'satisfaction_rating': 'offered',
  'status': 'closed',
  'total_seconds': 8657,
  'url': 'https://ambition.zendesk.com/api/v2/tickets/50740.json'},
 {'assignee_id': 389367924132,
  'created_at': '2021-